# Creating the Dataset for 2015

This Notebook: 
 - Reads in our All Public Schools data for 2015
 - Creates a new column for low performing schools 
 - Cleans up the dataset for ML capabilities

**The Final Datasets**: 
 - Includes whether school is low performing 
 - Removes Charter Schools
 
**Datasets ending in LPS_Processed are preprocessed for Machine Learning and go through the following transformations: **
1. Missing student body racial compositions are imputed using district averages.
2. Columns that have the same value in every single row are deleted.
3. Columns that have a unique value in every single row (all values are different) are deleted.
4. Empty columns (all values are NA or NULL) are deleted.
5. Numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
6. Remaining numeric, non-race columns with missing values are imputed / populated with 0.  In many cases, schools are not reporting values when they are zero. However, mean imputation or some other more sophisticated strategy might be considered here.
7. Categorical / text based columns with > *uniqueThreshold* unique values are deleted.
8. All remaining categorical / text based columns are one-hot encoded.  In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 

In [1]:
#import required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os

import seaborn as sns
sns.set(color_codes=True)

import warnings
warnings.filterwarnings("ignore")

Read in dataset from school year 2014-2015

In [2]:
#csv path
schoolYear = 2015

cwd = os.getcwd()
outputDir = cwd + '/'

publicSchools15 = pd.read_csv(outputDir + 'PublicSchools2015.csv', low_memory = False)
print(publicSchools15.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2585 entries, 0 to 2584
Columns: 362 entries, vphone_ad to WhitePct
dtypes: float64(315), int64(3), object(44)
memory usage: 7.1+ MB
None


### Low Performing School Definition:

“Low-performing schools are those that receive a school performance grade of D or F and a school growth score of "met expected growth" or "not met expected growth" as defined by G.S. 115C-83.15.” (G.S. 115C-105.37(a)), and

http://www.dpi.state.nc.us/schooltransformation/low-performing/

In [3]:
#create new column that equals 1 if low Performing
publicSchools15['LPS'] = np.where(publicSchools15['SPG Grade'].isin(['F', 'D'])&
                            publicSchools15['EVAAS Growth Status'].isin(['NotMet', 'Met']),
                            1, 0)

In [4]:
## Get the unit_codes of the low performing schools
unit_codes_15 = publicSchools15['unit_code'][publicSchools15.LPS == 1]

# Create Machine Learning Dataset for 2014

In [5]:
schoolData = publicSchools15.copy()

## Define Missing Data Thresholds
- Missing data threshold is 20% 
- Unique Categorical Data Threshold is 25

In [6]:
#Missing Data Threshold (Per Column)
missingThreshold = 0.20

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 25

print('*********Start: Beginning Column and Row Counts********************************************')
schoolData.info(verbose=False)

#Select only public schools as charter schools are missing data for many columns.
schoolData = schoolData[(schoolData['type_cd'] == 'P') & (schoolData['student_num'] > 0)]

print('\r\n*********After: Selecting Only Public School Campuses**********************************')
schoolData.info(verbose=False)

#Save primary key
unit_code = schoolData['unit_code']
#Convert zip code to string
schoolData['szip_ad'] = schoolData['szip_ad'].astype('object')
#Rename Lea_Name to District Name
schoolData.rename(columns={'Lea_Name': 'District Name'}, inplace=True)

*********Start: Beginning Column and Row Counts********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2585 entries, 0 to 2584
Columns: 363 entries, vphone_ad to LPS
dtypes: float64(315), int64(4), object(44)
memory usage: 7.2+ MB

*********After: Selecting Only Public School Campuses**********************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 363 entries, vphone_ad to LPS
dtypes: float64(315), int64(4), object(44)
memory usage: 6.8+ MB


## Student Body Racial Composition Features 
**Impute / update missing Student Body Racial Composition Fields using mean imputation.**
* When there are no racial composition percentages for a particular school campus / unit_code, fill in the missing values 

In [7]:
#Get Student Body Racial Composition Fields
raceCompositionFields = schoolData.filter(regex='Indian|Asian|Hispanic|Black|White|PacificIsland|TwoOrMore|Minority')\
                                  .filter(regex='Pct').columns
    
rowsBefore = schoolData[raceCompositionFields].isnull().T.any().T.sum()

#Update missing race values with the district average when avaiable (No district averages for charter schools) 
schoolData[raceCompositionFields] = schoolData.groupby('District Name')[raceCompositionFields]\
                                              .transform(lambda x: x.fillna(x.mean()))

    #Review dataset contents after Racial Composition Imputation
print('*********After: Updating Missing Racial Compostion Values****************************')   
rowsAfter = schoolData[raceCompositionFields].isnull().T.any().T.sum()
rowsUpdated = rowsBefore - rowsAfter
print ('Rows Updated / Imputed: ', rowsUpdated) 
print('\r\nTotal Rows Missing Racial Compositions By District Name') 
schoolData['District Name'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts()
print(schoolData['District Name'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts())

*********After: Updating Missing Racial Compostion Values****************************
Rows Updated / Imputed:  0

Total Rows Missing Racial Compositions By District Name
Series([], Name: District Name, dtype: int64)


## Remove Columns with Problematic Data
**Here we remove entire columns that could cause problems during machine learning.  The following operations are performed:**
* Remove any columns that have the same value in every single row.
* Remove any columns that have a unique value in every single row (all values are different).
* Remove empty columns (all values are NA or NULL).### Remove Columns with the same data in all rows

### Same Value in Every Single Row

In [8]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing fields with the same value in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(SingleValueCols))
print('Columns:', list(SingleValueCols))

*********After: Removing fields with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 244 entries, vphone_ad to LPS
dtypes: float64(203), int64(1), object(40)
memory usage: 4.6+ MB

Columns Deleted:  119
Columns: ['year', 'state_ad', 'type_cd', 'new_ind', 'super_nm', 'State_Name', 'stem', 'total_expense_num', 'salary_expense_pct', 'benefits_expense_pct', 'services_expense_pct', 'supplies_expense_pct', 'instruct_equip_exp_pct', 'other_expense_pct', 'federal_perpupil_num', 'local_perpupil_num', 'state_perpupil_num', 'lea_other_expense_pct', 'st_total_expense_num', 'st_salary_expense_pct', 'st_benefits_expense_pct', 'st_services_expense_pct', 'st_supplies_expense_pct', 'st_instruct_equip_exp_pct', 'st_other_expense_pct', 'st_federal_perpupil_num', 'st_local_perpupil_num', 'st_state_perpupil_num', 'building_expense_pct', 'lea_building_expense_pct', 'st_building_expense_pct', 'st_sat_avg_score_num', 'nat_sat_a

### Remove Columns with Unique Data In All Rows

In [9]:
#Remove any fields that have unique values in every row
schoolDataRecordCt = schoolData.shape[0]
UniqueValueCounts = schoolData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 242 entries, vphone_ad to LPS
dtypes: float64(203), int64(1), object(38)
memory usage: 4.6+ MB

Columns Deleted:  2


### Remove null/blank values in every row 

In [10]:
#Remove any empty fields (null values in every row)
schoolDataRecordCt = schoolData.shape[0]
NullValueCounts = schoolData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 242 entries, vphone_ad to LPS
dtypes: float64(203), int64(1), object(38)
memory usage: 4.5+ MB

Columns Deleted:  0


### Data Types

In [11]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print ("Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  38
Continuous Columns:  204
Columns Accounted for:  242


### Convert Booleans to 1s and 0s

In [12]:
#Convert Columns with values of nan, 'Y' as booleans
hasY = []
for x in schoolData.columns: 
    if ((['Y'] in schoolData[x].unique()) & (len(schoolData[x].unique()) == 2)):
        hasY.append(x)
print(hasY)

for x in hasY: 
# Map flag fields into bool
    schoolData[x] = schoolData[x].map({'Y':1, np.nan:0})

schoolData['esea_status'] = schoolData['esea_status'].map({'P':'Esea_Pass', 'F':'Esea_Fail', np.nan:'Non_Esea'})
schoolData['Grad_project_status'] = schoolData['Grad_project_status'].map({'Y':1, 'N':0, np.nan:0})

#Boolean Columns
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
print ("Boolean Columns: ", sD_boolean.shape[1])

['title1_type_cd', 'clp_ind', 'focus_clp_ind', 'summer_program_ind', 'asm_no_spg_ind', 'no_data_spg_ind']
Boolean Columns:  0


### Scale Expression of Percentages

In [13]:
PercentageFields = schoolData.filter(regex='pct|Pct|percent|Percent').columns
converted = []
for x in PercentageFields:
    if(schoolData[x].max() > 1):
        converted.append(x)
        schoolData[x] = schoolData[x]/100
print('Number of Percentage Fields:', len(PercentageFields))
print('Percentage Fields Converted to between 0 and 1: ', len(converted))

Number of Percentage Fields: 132
Percentage Fields Converted to between 0 and 1:  15


### Removal of STATE variables

In [14]:
stateFields = schoolData.filter(regex='st_').columns
print (stateFields)

keepFields = [col for col in schoolData.columns if col not in stateFields]
schoolData = schoolData[keepFields]
print(schoolData.info())

Index(['st_avg_student_num', 'st_total_specialized_courses',
       'st_ap_ib_courses', 'st_cte_courses', 'st_univ_college_courses',
       'st_avg_daily_attend_pct', 'st_crime_per_c_num',
       'st_short_susp_per_c_num', 'st_avg_age_media_collection',
       'st_books_per_student', 'st_flicensed_teach_pct',
       'st_tchyrs_0thru3_pct', 'st_tchyrs_4thru10_pct', 'st_tchyrs_11plus_pct',
       'st_class_teach_num', 'st_nbpts_num', 'st_advance_dgr_pct',
       'st_1yr_tchr_trnovr_pct', 'st_emer_prov_teach_pct',
       'st_lateral_teach_pct', 'st_highqual_class_pct'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 221 entries, vphone_ad to LPS
dtypes: float64(182), int64(8), object(31)
memory usage: 4.1+ MB
None


## Subset Data by Category before any more PreProcessing
By subsetting the dataset early, we avoid deleting columns that do not meet our missing data threshold because they may only be valid for certain schools (ie, graduate rate only collected for High Schools)

In [15]:
print('*********************************All Public Schools****************************')
schoolData.info(verbose=False)

#Filter regular public high schools
HighSchools = schoolData[((schoolData.category_cd == 'H') | 
                             (schoolData.category_cd == 'T') | 
                             (schoolData.category_cd == 'A')) &
                             (schoolData.student_num > 0) & 
                             (schoolData.school_type_txt == 'Regular School')
                            ]

print('*********************************Regular Public High Schools*******************')
HighSchools.info(verbose=False)

#Filter regular public middle schools
MiddleSchools = schoolData[((schoolData.category_cd == 'M') | 
                               (schoolData.category_cd == 'T') | 
                               (schoolData.category_cd == 'A') |
                               (schoolData.category_cd == 'I')) &
                               (schoolData.student_num > 0) & 
                               (schoolData.school_type_txt == 'Regular School')
                             ]

print('*********************************Regular Public Middle Schools******************')
MiddleSchools.info(verbose=False)


#Filter regular elementary high schools
ElementarySchools = schoolData[((schoolData.category_cd == 'E') | 
                                   (schoolData.category_cd == 'I') | 
                                   (schoolData.category_cd == 'A')) &
                                   (schoolData.student_num > 0) & 
                                   (schoolData.school_type_txt == 'Regular School')
                                 ]

print('*********************************Regular Public Elementary Schools**************')
ElementarySchools.info(verbose=False)

*********************************All Public Schools****************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 221 entries, vphone_ad to LPS
dtypes: float64(182), int64(8), object(31)
memory usage: 4.1+ MB
*********************************Regular Public High Schools*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 2583
Columns: 221 entries, vphone_ad to LPS
dtypes: float64(182), int64(8), object(31)
memory usage: 832.5+ KB
*********************************Regular Public Middle Schools******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 537 entries, 3 to 2581
Columns: 221 entries, vphone_ad to LPS
dtypes: float64(182), int64(8), object(31)
memory usage: 931.4+ KB
*********************************Regular Public Elementary Schools**************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1298 entries, 1 to 2584
Columns: 221 entries, vphone_ad to LPS
dtypes: float64(182), int64(8

## Isolate Datatypes for each subset

In [16]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal= schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print("*********** ALL SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print ("Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean_High = HighSchools.loc[:, (HighSchools.dtypes == bool) ]
sD_nominal_High = HighSchools.loc[:, (HighSchools.dtypes == object)]
sD_continuous_High = HighSchools.loc[:, (HighSchools.dtypes != bool) & (HighSchools.dtypes != object)]
print("*********** High SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean_High.shape[1])
print ("Nominal Columns: ", sD_nominal_High.shape[1])
print ("Continuous Columns: ", sD_continuous_High.shape[1])
print ("Columns Accounted for: ", sD_nominal_High.shape[1] + sD_continuous_High.shape[1] + sD_boolean_High.shape[1])

#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean_Middle = MiddleSchools.loc[:, (MiddleSchools.dtypes == bool) ]
sD_nominal_Middle = MiddleSchools.loc[:, (MiddleSchools.dtypes == object)]
sD_continuous_Middle = HighSchools.loc[:, (MiddleSchools.dtypes != bool) & (MiddleSchools.dtypes != object)]
print("*********** Middle SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean_Middle.shape[1])
print ("Nominal Columns: ", sD_nominal_Middle.shape[1])
print ("Continuous Columns: ", sD_continuous_Middle.shape[1])
print ("Columns Accounted for: ", sD_nominal_Middle.shape[1] + sD_continuous_Middle.shape[1] + sD_boolean_Middle.shape[1])

#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes == bool) ]
sD_nominal_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes == object)]
sD_continuous_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes != bool) & (ElementarySchools.dtypes != object)]
print("*********** Elementary SCHOOLS **********")
print ("Boolean Columns: ", sD_boolean_Elementary.shape[1])
print ("Nominal Columns: ", sD_nominal_Elementary.shape[1])
print ("Continuous Columns: ", sD_continuous_Elementary.shape[1])
print ("Columns Accounted for: ", sD_nominal_Elementary.shape[1] + sD_continuous_Elementary.shape[1] + sD_boolean_Elementary.shape[1])

*********** ALL SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  31
Continuous Columns:  190
Columns Accounted for:  221
*********** High SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  31
Continuous Columns:  190
Columns Accounted for:  221
*********** Middle SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  31
Continuous Columns:  190
Columns Accounted for:  221
*********** Elementary SCHOOLS **********
Boolean Columns:  0
Nominal Columns:  31
Continuous Columns:  190
Columns Accounted for:  221


### Eliminate columns with > 20% Data Missing

In [17]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* All Schools *************")
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))
print ('\r\nColumns: ', list(NullValueCols))

#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt_High = sD_continuous_High.shape[0]
missingValueLimit_High = schoolDataRecordCt_High * missingThreshold
NullValueCounts_High = sD_continuous_High.isnull().sum()
NullValueCols_High = NullValueCounts_High[NullValueCounts_High >= missingValueLimit].index
HighSchools = HighSchools.drop(NullValueCols_High, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* High Schools *************")
HighSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols_High))
print ('\r\nColumns: ', list(NullValueCols_High))

#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt_Middle = sD_continuous_Middle.shape[0]
missingValueLimit_Middle = schoolDataRecordCt_Middle * missingThreshold
NullValueCounts_Middle = sD_continuous_Middle.isnull().sum()
NullValueCols_Middle = NullValueCounts_Middle[NullValueCounts_Middle >= missingValueLimit].index
MiddleSchools = MiddleSchools.drop(NullValueCols_Middle, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* Middle Schools *************")
MiddleSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols_Middle))
print ('\r\nColumns: ', list(NullValueCols_Middle))

#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt_Elementary = sD_continuous_Elementary.shape[0]
missingValueLimit_Elementary = schoolDataRecordCt_Elementary * missingThreshold
NullValueCounts_Elementary = sD_continuous_Elementary.isnull().sum()
NullValueCols_Elementary = NullValueCounts_Elementary[NullValueCounts_Elementary >= missingValueLimit].index
ElementarySchools = ElementarySchools.drop(NullValueCols_Elementary, axis=1)

#Review dataset contents after empty field drops
print('\n')
print('*********After: Removing columns with >= missingThreshold % of missing values******')
print("************* Elementary Schools *************")
ElementarySchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols_Elementary))
print ('\r\nColumns: ', list(NullValueCols_Elementary))




*********After: Removing columns with >= missingThreshold % of missing values******
************* All Schools *************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 157 entries, vphone_ad to LPS
dtypes: float64(118), int64(8), object(31)
memory usage: 2.9+ MB

Columns Deleted:  64

Columns:  ['00_Size', '01_Size', '02_Size', '03_Size', '04_Size', '05_Size', '06_Size', '07_Size', '08_Size', 'Biology_Size', 'English II_Size', 'Math I_Size', 'Reading  SPG Score', 'Math SPG Score', 'Read Score', 'Math Score', 'Science Score', 'Math I Score', 'English II Score', 'Biology Score', 'The ACT Score', 'ACT WorkKeys Score', 'Math Course Rigor Score', 'Cohort Graduation Rate Standard Score', 'ACT_pTarget_PctMet', 'ACTWorkKeys_pTarget_PctMet', 'CurrentYearEOC_pTarget_PctMet', 'MathGr10_pTarget_PctMet', 'MathGr3-8_pTarget_PctMet', 'ReadingGr10_pTarget_PctMet', 'ReadingGr3-8_pTarget_PctMet', 'SciGr11_pTarget_PctMet', 'SciGr5&8_pTarget_PctMet', 'sat_avg_score

## Delete columns with more than 25 unique values before one hot encoding

In [18]:
#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))
print ('\r\nColumns : ', list(oneHotUniqueValueCols))

#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts_High = HighSchools[sD_nominal_High.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols_High = oneHotUniqueValueCounts_High[oneHotUniqueValueCounts_High >= uniqueThreshold].index
HighSchools.drop(oneHotUniqueValueCols_High, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('\n')
print('*********High Schools***********')
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
HighSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols_High))
print ('\r\nColumns : ', list(oneHotUniqueValueCols_High))

#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts_Middle = MiddleSchools[sD_nominal_Middle.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols_Middle = oneHotUniqueValueCounts_Middle[oneHotUniqueValueCounts_Middle >= uniqueThreshold].index
MiddleSchools.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('\n')
print('*********Middle Schools***********')
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
MiddleSchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols_Middle))
print ('\r\nColumns : ', list(oneHotUniqueValueCols_Middle))

#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts_Elementary = ElementarySchools[sD_nominal_Elementary.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols_Elementary = oneHotUniqueValueCounts_Elementary[oneHotUniqueValueCounts_Elementary >= uniqueThreshold].index
ElementarySchools.drop(oneHotUniqueValueCols_Elementary, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('\n')
print('*********Elementary Schools***********')
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
ElementarySchools.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols_Elementary))
print ('\r\nColumns : ', list(oneHotUniqueValueCols_Elementary))

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 143 entries, closed_ind to LPS
dtypes: float64(118), int64(8), object(17)
memory usage: 2.7+ MB

Columns Deleted:  14

Columns :  ['vphone_ad', 'street_ad', 'scity_ad', 'szip_ad', 'url_ad', 'grade_range_cd', 'cover_letter_ad', 'District Name', 'url', 'LEA Name', 'School Name', 'grades_BYOD', 'avg_age_media_collection', 'grades_1_to_1_access']


*********High Schools***********
*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 2583
Columns: 210 entries, closed_ind to LPS
dtypes: float64(182), int64(8), object(20)
memory usage: 791.2+ KB

Columns Deleted:  11

Columns :  ['vphone_ad', 'street_ad', 'scity_ad', 'szip_ad', 'url_ad', 'cover_letter_ad', 'District Name', 'url', 'LEA Name', 'School Name', 'avg_age_media_collection']



## One hot encode categorical variables

In [19]:
#Isolate remaining categorical variables
begColumnCt = len(schoolData.columns)
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]

#one hot encode categorical variables
schoolData = pd.get_dummies(data=schoolData, 
                       columns=sD_nominal, drop_first=True)

#Determine change in column count
endColumnCt = len(schoolData.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** All Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal.columns), list(sD_nominal.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
schoolData.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)


#Isolate remaining categorical variables
begColumnCt = len(HighSchools.columns)
sD_nominal_High = HighSchools.loc[:, (HighSchools.dtypes == object)]

#one hot encode categorical variables
HighSchools = pd.get_dummies(data=HighSchools, 
                       columns=sD_nominal_High, drop_first=True)

#Determine change in column count
endColumnCt = len(HighSchools.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** High Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal_High.columns), list(sD_nominal_High.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
HighSchools.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

#Isolate remaining categorical variables
begColumnCt = len(MiddleSchools.columns)
sD_nominal_Middle = MiddleSchools.loc[:, (MiddleSchools.dtypes == object)]

#one hot encode categorical variables
MiddleSchools = pd.get_dummies(data=MiddleSchools, 
                       columns=sD_nominal_Middle, drop_first=True)

#Determine change in column count
endColumnCt = len(MiddleSchools.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** Middle Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal_Middle.columns), list(sD_nominal_Middle.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
MiddleSchools.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

#Isolate remaining categorical variables
begColumnCt = len(ElementarySchools.columns)
sD_nominal_Elementary = ElementarySchools.loc[:, (ElementarySchools.dtypes == object)]

#one hot encode categorical variables
ElementarySchools = pd.get_dummies(data=ElementarySchools, 
                       columns=sD_nominal_Elementary, drop_first=True)

#Determine change in column count
endColumnCt = len(ElementarySchools.columns)
columnsAdded = endColumnCt - begColumnCt

#Review dataset contents one hot high unique value drops
print('\n')
print('*********** Elementary Schools ***********')
print ('Columns To One-Hot Encode: ', len(sD_nominal_Elementary.columns), list(sD_nominal_Elementary.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
ElementarySchools.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)




*********** All Schools ***********
Columns To One-Hot Encode:  17 ['category_cd', 'calendar_type_txt', 'sna_pgm_type_cd', 'school_type_txt', 'calendar_only_txt', 'esea_status', 'SBE District', 'SPG Grade', 'Reading SPG Grade', 'Math SPG Grade', 'EVAAS Growth Status', 'State Gap Compared', 'Category_Cd', 'esea_attendance', 'Byod', '_1_to_1_access', 'lea_avg_age_media_collection']

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Columns: 204 entries, closed_ind to lea_avg_age_media_collection_2010
dtypes: float64(118), int64(8), uint8(78)
memory usage: 2.5 MB

New Columns Created Via One-Hot Encoding:  61


*********** High Schools ***********
Columns To One-Hot Encode:  20 ['category_cd', 'grade_range_cd', 'calendar_type_txt', 'sna_pgm_type_cd', 'school_type_txt', 'calendar_only_txt', 'esea_status', 'SBE District', 'SPG Grade', 'Reading SPG Grade', 'Math SPG Grade', 'EVAAS Growt

### Impute remaining missing values to zero

In [20]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

#Check for Missing values 
missing_values_High = HighSchools.isnull().sum().reset_index()
missing_values_High.columns = ['Variable Name', 'Number Missing Values']
missing_values_High = missing_values_High[missing_values_High['Number Missing Values'] > 0] 
missing_values_High

#Check for Missing values 
missing_values_Middle = MiddleSchools.isnull().sum().reset_index()
missing_values_Middle.columns = ['Variable Name', 'Number Missing Values']
missing_values_Middle = missing_values_Middle[missing_values_Middle['Number Missing Values'] > 0] 
missing_values_Middle

#Check for Missing values 
missing_values_Elementary = ElementarySchools.isnull().sum().reset_index()
missing_values_Elementary.columns = ['Variable Name', 'Number Missing Values']
missing_values_Elementary = missing_values_Elementary[missing_values_Elementary['Number Missing Values'] > 0] 
missing_values_Elementary


*********The Remaining Missing Values Below will be set to Zero!*************************


,Variable Name,Number Missing Values
10,00_Size,111
11,01_Size,127
12,02_Size,126
13,03_Size,125
14,04_Size,102
15,05_Size,156
25,SPG Score,34
26,Reading SPG Score,36
27,Math SPG Score,36
28,EVAAS Growth Score,34


In [21]:
#Replace all remaining NaN with 0
schoolData = schoolData.fillna(0)
HighSchools = HighSchools.fillna(0)
MiddleSchools = MiddleSchools.fillna(0)
ElementarySchools = ElementarySchools.fillna(0)

In [22]:
#Restore the unit_code before saving
schoolData['unit_code'] = unit_code
HighSchools['unit_code'] = unit_code
MiddleSchools['unit_code'] = unit_code
ElementarySchools['unit_code'] = unit_code
#Save the final dataset to a .csv file
schoolData.to_csv(outputDir + 'PublicSchools' + str(schoolYear) + '_LPS_Processed' + '.csv', sep=',', index=False)
HighSchools.to_csv(outputDir + 'PublicHighSchools' + str(schoolYear) + '_LPS_Processed' + '.csv', sep=',', index=False)
MiddleSchools.to_csv(outputDir + 'PublicMiddleSchools' + str(schoolYear) + '_LPS_Processed' + '.csv', sep=',', index=False)
ElementarySchools.to_csv(outputDir + 'PublicElementarySchools' + str(schoolYear) + '_LPS_Processed' +'.csv', sep=',', index=False)


In [23]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
schoolData.info(verbose=True)
HighSchools.info(verbose=True)
MiddleSchools.info(verbose=True)
ElementarySchools.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2432 entries, 0 to 2584
Data columns (total 205 columns):
closed_ind                                                                         float64
title1_type_cd                                                                     int64
clp_ind                                                                            int64
focus_clp_ind                                                                      int64
summer_program_ind                                                                 int64
asm_no_spg_ind                                                                     int64
no_data_spg_ind                                                                    int64
student_num                                                                        float64
lea_avg_student_num                                                                floa